In [1]:
#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("chanyoongqiedison/training")

#print("Path to dataset files:", path)

In [2]:
# cnn_transformer_colorizer_v1.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNEncoder(nn.Module):
    def __init__(self, in_channels=1, feature_dim=256):
        super().__init__()
        # Store intermediate features for skip connections
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, feature_dim, 3, stride=1, padding=1),
            nn.BatchNorm2d(feature_dim),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x1 = self.conv1(x)  # [B, 64, H, W]
        x2 = self.conv2(x1) # [B, 128, H/2, W/2]
        x3 = self.conv3(x2) # [B, 256, H/4, W/4]
        x4 = self.conv4(x3) # [B, 256, H/4, W/4]
        return x4, (x1, x2, x3)  # Return features for skip connections


class TransformerEncoder(nn.Module):
    def __init__(self, feature_dim=256, num_heads=8, num_layers=4, patch_size=4):
        super().__init__()
        self.feature_dim = feature_dim
        self.patch_size = patch_size
        
        # Calculate projection dimension (patch_size^2 * feature_dim)
        patch_dim = patch_size * patch_size * feature_dim
        
        # Create projection layer as a module (will be trained)
        self.patch_projection = nn.Linear(patch_dim, feature_dim)
        
        # Positional embedding (will be adjusted dynamically if needed)
        self.pos_embed = nn.Parameter(torch.randn(1, 256, feature_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=feature_dim,
            nhead=num_heads,
            dim_feedforward=feature_dim * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, x):
        B, C, H, W = x.shape

        # Split feature map into patches
        x = x.unfold(2, self.patch_size, self.patch_size)\
             .unfold(3, self.patch_size, self.patch_size)
        x = x.contiguous().view(B, C, -1, self.patch_size * self.patch_size)
        x = x.permute(0, 2, 1, 3).flatten(2)  # [B, N_patches, C * patch_area]

        # Project patches to feature dimension
        x = self.patch_projection(x)  # [B, N_patches, feature_dim]
        
        # Add positional embedding
        num_patches = x.size(1)
        if num_patches <= self.pos_embed.size(1):
            pos_emb = self.pos_embed[:, :num_patches, :]
        else:
            # Interpolate if we have more patches than expected
            pos_emb = F.interpolate(
                self.pos_embed.permute(0, 2, 1),
                size=num_patches,
                mode='linear',
                align_corners=False
            ).permute(0, 2, 1)
        
        x = x + pos_emb

        # Transformer processing
        x = self.transformer(x)
        return x, (H, W)  # Return original spatial dims


class ColorDecoder(nn.Module):
    def __init__(self, feature_dim=256):
        super().__init__()
        # Enhanced decoder with more capacity and skip connections
        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(feature_dim, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        
        self.up2 = nn.Sequential(
            nn.ConvTranspose2d(256 + 256, 128, 4, stride=2, padding=1),  # + skip from encoder
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        
        self.up3 = nn.Sequential(
            nn.Conv2d(128 + 128, 128, 3, stride=1, padding=1),  # + skip from encoder
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        
        self.final = nn.Sequential(
            nn.Conv2d(128 + 64, 64, 3, stride=1, padding=1),  # + skip from encoder
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, 3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 2, 3, stride=1, padding=1),
            nn.Tanh()  # final a,b in [-1,1]
        )

    def forward(self, x, orig_H, orig_W, skip_features=None):
        """
        Args:
            x: [B, N_patches, feature_dim]
            orig_H, orig_W: Original input image dimensions
            skip_features: Tuple of (x1, x2, x3) from encoder
        """
        B, N, C = x.shape
        side = int(N ** 0.5)
        
        # Reshape patches back to spatial grid
        x = x.permute(0, 2, 1).view(B, C, side, side)  # [B, feature_dim, H', W']

        # Apply decoder upsampling with skip connections
        x = self.up1(x)  # 2x upsampling
        
        if skip_features is not None:
            # Add skip connection from encoder's conv3 (after proper sizing)
            skip3 = skip_features[2]  # [B, 256, H/4, W/4]
            if skip3.shape[2:] != x.shape[2:]:
                skip3 = F.interpolate(skip3, size=x.shape[2:], mode='bilinear', align_corners=False)
            x = torch.cat([x, skip3], dim=1)
        
        x = self.up2(x)  # 4x upsampling total
        
        if skip_features is not None:
            # Add skip connection from encoder's conv2
            skip2 = skip_features[1]  # [B, 128, H/2, W/2]
            if skip2.shape[2:] != x.shape[2:]:
                skip2 = F.interpolate(skip2, size=x.shape[2:], mode='bilinear', align_corners=False)
            x = torch.cat([x, skip2], dim=1)
        
        x = self.up3(x)
        
        if skip_features is not None:
            # Add skip connection from encoder's conv1
            skip1 = skip_features[0]  # [B, 64, H, W]
            if skip1.shape[2:] != x.shape[2:]:
                skip1 = F.interpolate(skip1, size=x.shape[2:], mode='bilinear', align_corners=False)
            x = torch.cat([x, skip1], dim=1)
        
        x = self.final(x)
        
        # Final resize to exact target dimensions
        if x.shape[2] != orig_H or x.shape[3] != orig_W:
            x = F.interpolate(x, size=(orig_H, orig_W), mode="bilinear", align_corners=False)
        
        return x


class CNNTransformerColorizer(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = CNNEncoder()
        self.transformer = TransformerEncoder()
        self.decoder = ColorDecoder()

    def forward(self, x):
        orig_H, orig_W = x.shape[2], x.shape[3]
        
        # Encode with skip connections
        feat, skip_features = self.encoder(x)  # [B, 256, H/4, W/4]
        
        # Transform
        trans, (enc_H, enc_W) = self.transformer(feat)  # [B, N_patches, 256]
        
        # Decode with skip connections
        ab = self.decoder(trans, orig_H, orig_W, skip_features)  # [B, 2, H, W]
        
        return ab

In [3]:
# train.py
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from pathlib import Path
from PIL import Image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import torch.nn.functional as F
#from cnn_transformer_colorizer_v1 import CNNTransformerColorizer 

# ============================================================
# Enhanced Loss Function
# ============================================================
class EnhancedColorizationLoss(nn.Module):
    def __init__(self, l1_weight=1.0, color_weight=0.3, saturation_weight=0.2):
        super().__init__()
        self.l1_weight = l1_weight
        self.color_weight = color_weight
        self.saturation_weight = saturation_weight
        self.l1_loss = nn.L1Loss()
        
    def color_distribution_loss(self, pred, target):
        # Encourage similar color distribution using histogram matching
        pred_flat = pred.view(pred.size(0), 2, -1)
        target_flat = target.view(target.size(0), 2, -1)
        
        # Compute histogram for each channel
        hist_loss = 0
        for c in range(2):
            pred_hist = torch.histc(pred_flat[:, c], bins=64, min=-1, max=1)
            target_hist = torch.histc(target_flat[:, c], bins=64, min=-1, max=1)
            hist_loss += F.l1_loss(pred_hist, target_hist)
        
        return hist_loss / 2
    
    def saturation_encouragement(self, pred):
        # Encourage higher saturation (penalize gray outputs)
        # Compute saturation as magnitude in AB space
        saturation = torch.sqrt(pred[:, 0:1]**2 + pred[:, 1:2]**2)
        # We want to maximize saturation, so we minimize negative saturation
        return -torch.mean(saturation)
    
    def forward(self, pred, target):
        l1_loss = self.l1_loss(pred, target)
        color_loss = self.color_distribution_loss(pred, target)
        sat_loss = self.saturation_encouragement(pred)
        
        total_loss = (self.l1_weight * l1_loss + 
                     self.color_weight * color_loss + 
                     self.saturation_weight * sat_loss)
        
        return total_loss, {'l1': l1_loss.item(), 'color': color_loss.item(), 'saturation': sat_loss.item()}

# ============================================================
# Dataset
# ============================================================
class ColorizationDataset(Dataset):
    def __init__(self, data_dir, split="train", max_samples=100000):
        data_dir = Path(data_dir)

        # Correct structure:
        # processed/grayscale/train/*.jpg
        # processed/color/train/*.npy
        g_root = data_dir / "grayscale" / split
        c_root = data_dir / "color" / split

        if not g_root.exists() or not c_root.exists():
            raise RuntimeError(
                f"Expected folders:\n{g_root}\n{c_root}\nbut they were not found."
            )

        print(f"📁 Loading {split.upper()} split")
        print(f"Searching for grayscale images in: {g_root}")

        # Load grayscale files
        self.gray_files = []
        for root, dirs, files in os.walk(g_root):
            jpg_files = [f for f in files if f.lower().endswith(".jpg")]
            for f in jpg_files:
                self.gray_files.append(Path(root) / f)
                if len(self.gray_files) >= max_samples:
                    break
            if len(self.gray_files) >= max_samples:
                break

        print(f"📁 Total grayscale images found: {len(self.gray_files)}")

        self.gray_files = sorted(self.gray_files)
        self.gray_root = g_root
        self.color_root = c_root

    def __len__(self):
        return len(self.gray_files)

    def __getitem__(self, idx):
        gray_path = self.gray_files[idx]

        # relative path inside grayscale/train
        rel = gray_path.relative_to(self.gray_root)

        # corresponding color/train/*.npy
        color_path = (self.color_root / rel).with_suffix(".npy")

        # Load grayscale
        L = np.array(Image.open(gray_path)).astype("float32")
        L = (L / 127.5) - 1.0

        # Load AB
        AB = np.load(color_path).astype("float32")

        L = torch.tensor(L).unsqueeze(0)
        AB = torch.tensor(AB).permute(2, 0, 1)

        return L, AB



# ============================================================
# Enhanced Training Function with Checkpoints and Visualizations
# ============================================================
def train_model_multi_gpu(model, train_loader, criterion, optimizer, scheduler, device, epochs=5, save_dir="/kaggle/working"):
    """
    Enhanced training function with periodic checkpoints and visualizations
    """
    model.train()
    
    # Create save directory
    save_path = Path(save_dir)
    save_path.mkdir(exist_ok=True)
    
    # Store loss history
    loss_history = {
        'total': [], 'l1': [], 'color': [], 'saturation': [], 'lr': []
    }
    
    # GPU monitoring function
    def print_gpu_usage():
        if torch.cuda.is_available():
            print("GPU Memory Usage:")
            for i in range(torch.cuda.device_count()):
                alloc = torch.cuda.memory_allocated(i) / 1024**3
                cached = torch.cuda.memory_reserved(i) / 1024**3
                print(f"  GPU {i}: {alloc:.2f}GB / {cached:.2f}GB")

    for epoch in range(epochs):
        total_loss = 0.0
        total_l1 = 0.0
        total_color = 0.0
        total_sat = 0.0
        
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}")

        for i, (L, AB) in pbar:
            L, AB = L.to(device, non_blocking=True), AB.to(device, non_blocking=True)

            # Forward pass
            optimizer.zero_grad()
            output = model(L)

            # --- Sanity check (only for first batch of first epoch)
            if epoch == 0 and i == 0:
                print("\n=== Sanity check for shapes ===")
                print(f"Input L shape: {L.shape}")
                print(f"Output (predicted AB) shape: {output.shape}")
                print(f"Target AB shape: {AB.shape}")
                print(f"Output min/max: {output.min().item():.4f} / {output.max().item():.4f}")
                print(f"Target min/max: {AB.min().item():.4f} / {AB.max().item():.4f}")

            # --- Resize for safety on every batch
            if output.shape != AB.shape:
                output = torch.nn.functional.interpolate(output, size=(AB.shape[2], AB.shape[3]), mode='bilinear', align_corners=False)

            # Compute enhanced loss
            loss, loss_components = criterion(output, AB)

            # Backward pass
            loss.backward()
            
            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            total_l1 += loss_components['l1']
            total_color += loss_components['color']
            total_sat += loss_components['saturation']
            
            pbar.set_postfix({
                'loss': loss.item(), 
                'l1': loss_components['l1'],
                'color': loss_components['color']
            })
            
            # Print GPU usage first batch of first epoch
            if epoch == 0 and i == 0:
                print_gpu_usage()

        # Step the scheduler
        if scheduler is not None:
            scheduler.step()
            current_lr = scheduler.get_last_lr()[0]
        else:
            current_lr = optimizer.param_groups[0]['lr']
            
        avg_loss = total_loss / len(train_loader)
        avg_l1 = total_l1 / len(train_loader)
        avg_color = total_color / len(train_loader)
        avg_sat = total_sat / len(train_loader)
        
        # Store loss history
        loss_history['total'].append(avg_loss)
        loss_history['l1'].append(avg_l1)
        loss_history['color'].append(avg_color)
        loss_history['saturation'].append(avg_sat)
        loss_history['lr'].append(current_lr)
        
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.6f} (L1: {avg_l1:.6f}, Color: {avg_color:.6f}, Sat: {avg_sat:.6f})")
        print(f"Learning Rate: {current_lr:.2e}")

        # ============================================================
        # Save checkpoint and generate visualization every 20 epochs
        # ============================================================
        if (epoch + 1) % 20 == 0 or epoch == 0 or epoch == epochs - 1:
            print(f"\n💾 Checkpoint at epoch {epoch+1}")
            
            # Handle DataParallel wrapping for saving
            if hasattr(model, 'module'):
                model_to_save = model.module
            else:
                model_to_save = model
            
            # Save model checkpoint
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model_to_save.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
                'loss_history': loss_history,
                'total_loss': avg_loss,
            }
            
            checkpoint_path = save_path / f"checkpoint_epoch_{epoch+1:03d}.pth"
            torch.save(checkpoint, checkpoint_path)
            print(f"✅ Checkpoint saved: {checkpoint_path}")
            
            # Generate and save visualization
            print("🎨 Generating visualization...")
            try:
                visualization_path = save_path / f"visualization_epoch_{epoch+1:03d}.png"
                generate_visualization(model_to_save, train_loader.dataset, device, 
                                     save_path=visualization_path, epoch=epoch+1)
                print(f"✅ Visualization saved: {visualization_path}")
            except Exception as e:
                print(f"❌ Failed to generate visualization: {e}")
            
            # Plot loss history
            plot_loss_history(loss_history, save_path / f"loss_plot_epoch_{epoch+1:03d}.png")
            print(f"📊 Loss plot saved")
            
            print("=" * 60)

    print("Training completed successfully!")
    
    # Save final model
    if hasattr(model, 'module'):
        model_to_save = model.module
    else:
        model_to_save = model
        
    final_model_path = save_path / "final_model.pth"
    torch.save(model_to_save.state_dict(), final_model_path)
    print(f"✅ Final model saved: {final_model_path}")
    
    return model, loss_history


# ============================================================
# Enhanced Visualization with Saving Capability
# ============================================================
def generate_visualization(model, dataset, device, num_samples=3, saturation_boost=1.3, 
                         save_path=None, epoch=None):
    """
    Visualize model predictions and optionally save to file.
    All values are in [-1, 1] range.
    """
    from skimage import color
    
    model.eval()
    idxs = np.random.choice(len(dataset), num_samples, replace=False)
    
    fig = plt.figure(figsize=(15, 4 * num_samples))
    
    if epoch is not None:
        plt.suptitle(f"Epoch {epoch} - Model Predictions", fontsize=16, y=0.95)
    
    for i, idx in enumerate(idxs):
        L, AB = dataset[idx]
        
        # Get prediction
        with torch.no_grad():
            pred_AB = model(L.unsqueeze(0).to(device)).cpu().squeeze(0)
        
        # Apply saturation boost to predictions
        pred_AB_boosted = pred_AB * saturation_boost
        pred_AB_boosted = torch.clamp(pred_AB_boosted, -1.0, 1.0)
        
        # Convert from tensors to numpy
        L_np = L.squeeze().numpy()  # [-1, 1]
        pred_AB_np = pred_AB.permute(1, 2, 0).numpy()  # [-1, 1]
        pred_AB_boosted_np = pred_AB_boosted.permute(1, 2, 0).numpy()  # [-1, 1]
        AB_gt_np = AB.permute(1, 2, 0).numpy()  # [-1, 1]
        
        # Convert back to CIELAB range
        L_lab = (L_np + 1.0) * 50.0  # [-1, 1] -> [0, 100]
        pred_AB_lab = pred_AB_np * 110.0  # [-1, 1] -> [-110, 110]
        pred_AB_boosted_lab = pred_AB_boosted_np * 110.0  # [-1, 1] -> [-110, 110]
        AB_gt_lab = AB_gt_np * 110.0  # [-1, 1] -> [-110, 110]
        
        # Reconstruct LAB images
        pred_lab = np.zeros((L_lab.shape[0], L_lab.shape[1], 3))
        pred_lab[:, :, 0] = L_lab
        pred_lab[:, :, 1:] = pred_AB_lab
        
        pred_boosted_lab = np.zeros_like(pred_lab)
        pred_boosted_lab[:, :, 0] = L_lab
        pred_boosted_lab[:, :, 1:] = pred_AB_boosted_lab
        
        gt_lab = np.zeros_like(pred_lab)
        gt_lab[:, :, 0] = L_lab
        gt_lab[:, :, 1:] = AB_gt_lab
        
        # Convert LAB to RGB using skimage
        pred_rgb = color.lab2rgb(pred_lab)
        pred_boosted_rgb = color.lab2rgb(pred_boosted_lab)
        gt_rgb = color.lab2rgb(gt_lab)
        
        # Plot
        plt.subplot(num_samples, 4, 4 * i + 1)
        plt.imshow(L_np, cmap="gray", vmin=-1, vmax=1)
        plt.title("Input L")
        plt.axis("off")

        plt.subplot(num_samples, 4, 4 * i + 2)
        plt.imshow(np.clip(pred_rgb, 0, 1))
        plt.title("Predicted Color")
        plt.axis("off")
        
        plt.subplot(num_samples, 4, 4 * i + 3)
        plt.imshow(np.clip(pred_boosted_rgb, 0, 1))
        plt.title(f"Boosted (x{saturation_boost})")
        plt.axis("off")

        plt.subplot(num_samples, 4, 4 * i + 4)
        plt.imshow(np.clip(gt_rgb, 0, 1))
        plt.title("Ground Truth")
        plt.axis("off")
        
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close(fig)  # Close the figure to free memory
    else:
        plt.show()


# ============================================================
# NEW: Loss History Plotting Function
# ============================================================
def plot_loss_history(loss_history, save_path=None):
    """
    Plot and save loss history
    """
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    epochs = range(1, len(loss_history['total']) + 1)
    
    # Total loss
    ax1.plot(epochs, loss_history['total'])
    ax1.set_title('Total Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.grid(True, alpha=0.3)
    
    # L1 loss
    ax2.plot(epochs, loss_history['l1'])
    ax2.set_title('L1 Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('L1 Loss')
    ax2.grid(True, alpha=0.3)
    
    # Color loss
    ax3.plot(epochs, loss_history['color'])
    ax3.set_title('Color Distribution Loss')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Color Loss')
    ax3.grid(True, alpha=0.3)
    
    # Learning rate
    ax4.plot(epochs, loss_history['lr'])
    ax4.set_title('Learning Rate')
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Learning Rate')
    ax4.set_yscale('log')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()


# ============================================================
# Auto-detect Dataset Path (unchanged)
# ============================================================
def find_dataset_path():
    """Automatically find the dataset path in Kaggle"""
    input_dir = Path("/kaggle/input/100k-data")
    
    if not input_dir.exists():
        print("/kaggle/input directory not found")
        return None
    
    available_datasets = [d for d in input_dir.iterdir() if d.is_dir()]
    print(f"Available datasets: {[d.name for d in available_datasets]}")
    
    # Look for processed data
    for dataset in available_datasets:
        potential_path = dataset / "processed"
        if potential_path.exists():
            print(f"Found processed data at: {potential_path}")
            return str(potential_path)
    
    # Look for train/grayscale structure directly
    for dataset in available_datasets:
        potential_train = dataset / "train" / "grayscale" # change "val" back to "train", mismatched  folder names
        if potential_train.exists():
            print(f"Found data at: {dataset}")
            return str(dataset)
    
    # Use the first dataset as fallback
    if available_datasets:
        fallback = available_datasets[0]
        print(f"Using fallback dataset: {fallback}")
        return str(fallback)
    
    print("No datasets found")
    return None


# ============================================================
# Enhanced Main Function with Multi-GPU Support
# ============================================================
def main():
    # GPU configuration
    num_gpus = torch.cuda.device_count()
    print(f"Found {num_gpus} GPU(s)")
    
    for i in range(num_gpus):
        gpu_props = torch.cuda.get_device_properties(i)
        print(f"  - GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Memory: {gpu_props.total_memory / 1024**3:.1f} GB")
    
    # Use all available GPUs
    if num_gpus > 1:
        device = torch.device("cuda:0")  # Use first GPU as main
        print(f"Using {num_gpus} GPUs with DataParallel")
        
        # Model with DataParallel
        model = CNNTransformerColorizer()
        model = DataParallel(model)
        model = model.to(device)
        
        # Larger batch size for multiple GPUs
        batch_size = 8 * num_gpus
        
    else:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = CNNTransformerColorizer().to(device)
        batch_size = 8
        print(f"Using device: {device}")

    # Auto-detect dataset path for Kaggle
    data_dir = find_dataset_path()
    if data_dir is None:
        print("Could not find dataset. Please check your Kaggle dataset.")
        return
    
    print(f"Using data directory: {data_dir}")
    
    try:
        train_dataset = ColorizationDataset(data_dir, split="train", max_samples=10000)
        train_loader = DataLoader(
            train_dataset, 
            batch_size=batch_size, 
            shuffle=True, 
            num_workers=2,
            pin_memory=True
        )
        print(f"Training with {len(train_dataset)} images")
        print(f"Batch size: {batch_size} (adjusted for {num_gpus} GPU(s))")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        print("Available directories:")
        for root, dirs, files in os.walk(data_dir):
            level = root.replace(str(data_dir), '').count(os.sep)
            indent = ' ' * 2 * level
            print(f'{indent}{os.path.basename(root)}/')
            subindent = ' ' * 2 * (level + 1)
            for file in files[:5]:
                print(f'{subindent}{file}')
            if len(files) > 5:
                print(f'{subindent}... and {len(files) - 5} more files')
        return

    # Enhanced loss function and optimizer
    criterion = EnhancedColorizationLoss(l1_weight=1.0, color_weight=0.3, saturation_weight=0.2)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    
    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-6)

    print("Starting training with enhanced colorization...")
    print("Checkpoints and visualizations will be saved every 20 epochs")
    print("=" * 60)
    
    # Train with checkpointing and visualization
    model, loss_history = train_model_multi_gpu(
        model, train_loader, criterion, optimizer, scheduler, device, 
        epochs=200, save_dir="/kaggle/working"
    )

    print("🎉 Training completed successfully!")
    print(f"📁 All checkpoints and visualizations saved to: /kaggle/working/")


if __name__ == "__main__":
    main()

Found 2 GPU(s)
  - GPU 0: Tesla T4
    Memory: 14.7 GB
  - GPU 1: Tesla T4
    Memory: 14.7 GB
Using 2 GPUs with DataParallel
Available datasets: ['processed']
Using fallback dataset: /kaggle/input/100k-data/processed
Using data directory: /kaggle/input/100k-data/processed
📁 Loading TRAIN split
Searching for grayscale images in: /kaggle/input/100k-data/processed/grayscale/train
📁 Total grayscale images found: 10000
Training with 10000 images
Batch size: 16 (adjusted for 2 GPU(s))
Starting training with enhanced colorization...
Checkpoints and visualizations will be saved every 20 epochs


Epoch 1/200:   0%|          | 0/625 [00:00<?, ?it/s]


=== Sanity check for shapes ===
Input L shape: torch.Size([16, 1, 256, 256])
Output (predicted AB) shape: torch.Size([16, 2, 256, 256])
Target AB shape: torch.Size([16, 2, 256, 256])
Output min/max: -0.7445 / 0.9611
Target min/max: -0.7138 / 0.7822


Epoch 1/200:   0%|          | 2/625 [00:04<19:23,  1.87s/it, loss=6.01e+3, l1=0.376, color=2e+4]

GPU Memory Usage:
  GPU 0: 0.15GB / 1.24GB
  GPU 1: 0.01GB / 1.49GB


...

Epoch [198/200] - Loss: 4221.684548 (L1: 0.076636, Color: 14072.085075, Sat: -0.088991)
Learning Rate: 9.99e-04


Epoch 199/200: 100%|██████████| 625/625 [01:36<00:00,  6.45it/s, loss=4.25e+3, l1=0.065, color=1.42e+4]

Epoch [199/200] - Loss: 4204.847917 (L1: 0.076643, Color: 14015.962775, Sat: -0.088677)
Learning Rate: 1.00e-03



Epoch 200/200: 100%|██████████| 625/625 [01:36<00:00,  6.47it/s, loss=4.67e+3, l1=0.0746, color=1.56e+4]


Epoch [200/200] - Loss: 4217.089261 (L1: 0.076751, Color: 14056.767175, Sat: -0.089092)
Learning Rate: 1.00e-03

💾 Checkpoint at epoch 200
✅ Checkpoint saved: /kaggle/working/checkpoint_epoch_200.pth
🎨 Generating visualization...
✅ Visualization saved: /kaggle/working/visualization_epoch_200.png
📊 Loss plot saved
Training completed successfully!
✅ Final model saved: /kaggle/working/final_model.pth
🎉 Training completed successfully!
📁 All checkpoints and visualizations saved to: /kaggle/working/


In [4]:
"""import os
import shutil

folder_path = '/kaggle/working'
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')"""

"import os\nimport shutil\n\nfolder_path = '/kaggle/working'\nfor filename in os.listdir(folder_path):\n    file_path = os.path.join(folder_path, filename)\n    try:\n        if os.path.isfile(file_path) or os.path.islink(file_path):\n            os.unlink(file_path)\n        elif os.path.isdir(file_path):\n            shutil.rmtree(file_path)\n    except Exception as e:\n        print(f'Failed to delete {file_path}. Reason: {e}')"